---

### *Title: Estimating the Parameters of the Bass Model*
Subtitle: Using Solver to Find the Best Fit for P, Q, and N
Content:
• How can we determine the optimal values for the Bass model parameters?
   - Use Solver to find P, Q, and N that minimize the Sum of Squared Errors (SSE)
      1. Input trial values for P, Q, and N
      2. Calculate predicted sales using the Bass equation
      3. Compute SSE between actual and predicted sales
   - Solver adjusts P, Q, and N to minimize SSE
      - Constraints: 0 ≤ P ≤ 1, 0 ≤ Q ≤ 1
   - Relative magnitudes of P and Q indicate importance of innovation vs. imitation


### *Title: Estimating Bass Model Parameters in Excel*
Content:
1. In cells E2:G2, input initial guesses for P, Q, and N. Name these cells p, q, and Nbar.

2. In cell D6, enter the Bass model formula: 
   =p*(Nbar-C5)+(q/Nbar)*C5*(Nbar-C5)
   Copy this formula down to cells D7:D21.

3. In cell E6, calculate the squared error between predicted and actual sales:
   =(B6-D6)^2 
   Copy this formula down to cells E7:E21.

4. In cell E3, calculate the Sum of Squared Errors (SSE) using:
   =SUM(E6:E21)

5. Open Solver and set it up as follows:
   - Set Objective: E3
   - To: Min
   - By Changing Variable Cells: $E$2:$G$2
   - Subject to the Constraints:
      - $E$2:$F$2 >= 0
      - $E$2:$F$2 <= 1
   - Select GRG Nonlinear as the Solving Method
   - Click Solve to find the optimal values of P, Q, and N that minimize SSE.


---

This Python code does the following:

1. Imports necessary libraries (pandas, sqlite3, numpy, scipy.optimize)
2. Loads sales data from the Excel file "ColorTV.xlsx" into a pandas DataFrame
3. Creates an SQLite database "ColorTV.db" and a table "sales_data" from the DataFrame
4. Retrieves data from the SQLite table into a new DataFrame for further analysis
5. Defines the Bass model equation and the cost function (Sum of Squared Errors) to minimize
6. Sets initial parameter guesses and bounds for P, Q, and M
7. Uses scipy.optimize.minimize to find the optimal parameter values that minimize SSE
8. Prints out the optimal values of P, Q, and M
9. Closes the SQLite database connection

The code leverages SQLite to store and retrieve data efficiently, and utilizes powerful Python libraries to perform the optimization and estimation of Bass model parameters.

---

In [ ]:
import pandas as pd
import sqlite3
import numpy as np
from scipy.optimize import minimize

# Load data from Excel into a pandas DataFrame
data = pd.read_excel('../data/ColorTV.xlsx', sheet_name='Sheet1', usecols='A:C', skiprows=4, nrows=16)

# Connect to SQLite database and create a table from the DataFrame
conn = sqlite3.connect('ColorTV.db')
data.to_sql('sales_data', conn, if_exists='replace', index=False)

# Retrieve data from SQLite into a new DataFrame
query = 'SELECT * FROM sales_data'
df = pd.read_sql(query, conn)

# Define the Bass model equation
def bass_model(t, p, q, m):
    a = p + q
    b = p / m
    c = np.exp(-(a * t))
    return m * (((a**2)/p) * c) / (1 + ((q/p) * c))**2

# Define the cost function to minimize (Sum of Squared Errors)
def cost_function(params):
    p, q, m = params
    y_pred = bass_model(df['t'], p, q, m)
    y_true = df['n(t)']
    sse = np.sum((y_true - y_pred)**2)
    return sse

# Set initial parameter guesses and bounds
initial_params = [0.03, 0.38, 100]
bounds = [(0, 1), (0, 1), (0, None)]

# Use scipy.optimize.minimize to find optimal parameters
result = minimize(cost_function, initial_params, bounds=bounds, method='L-BFGS-B')
p, q, m = result.x

# Print optimal parameter values
print(f'Optimal P: {p:.3f}')  
print(f'Optimal Q: {q:.3f}')
print(f'Optimal M: {m:.3f}')

# Close the SQLite connection
conn.close()